In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, classification_report, log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
import time

In [2]:
def esa_score(phi, alpha):
    return np.dot(alpha, phi)

def threshold_crossing_rate(esa_baseline, esa_moral, tau):
    crossed = (esa_baseline < tau) & (esa_moral >= tau)
    return np.mean(crossed)

def moral_win_rate(esa_baseline, esa_moral):
    return np.mean(esa_moral > esa_baseline)

def esa_difference(esa_baseline, esa_moral):
    return np.mean(esa_moral - esa_baseline)

In [3]:
def evaluate_classification(y_true, y_pred):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1_score': f1_score(y_true, y_pred),
        'roc_auc': roc_auc_score(y_true, y_pred)
    }
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    return metrics

In [6]:
df = pd.read_csv('grad admission - ethics.csv')
    
y = df['accept_status_moral']
X = df.drop(columns=['accept_status', 'ESA', 'CST', 'severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int', 'accept_status_moral'])

esa_features = df[['severity_cons','dur_cons','util_cons','prin_up','prin_vi','moral_int']].values
tau_values = df['CST'].values
alpha = np.array([0.4, 0.2, 0.3, 0.0, 0.0, 0.1])


In [11]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

nb_baseline_preds = np.zeros(len(X))
nb_override_preds = np.zeros(len(X))
nb_penalized_preds = np.zeros(len(X))

b_time = 0
o_time = 0
p_time = 0

for fold, (train_idx, val_idx) in enumerate(skf.split(X_scaled, y)):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    b_time_s = time.perf_counter()
    # -------- Baseline --------
    model_baseline = GaussianNB()
    model_baseline.fit(X_train, y_train)
    y_val_pred_baseline = model_baseline.predict(X_val)
    nb_baseline_preds[val_idx] = y_val_pred_baseline
    print(f"Fold {fold+1} - Naive Bayes Baseline:")
    evaluate_classification(y_val, y_val_pred_baseline)
    b_time_e = time.perf_counter()
    b_time = b_time + (b_time_e - b_time_s)
    
    o_time_s = time.perf_counter()
    # -------- Override --------
    phi_val = esa_features[val_idx]
    tau_val = tau_values[val_idx]
    esa_vals = np.array([esa_score(phi, alpha) for phi in phi_val])
    moral_preds = (esa_vals >= tau_val).astype(int)
    nb_override_preds[val_idx] = moral_preds
    print(f"Fold {fold+1} - Naive Bayes ESA Override:")
    evaluate_classification(y_val, moral_preds)
    o_time_e = time.perf_counter()
    o_time = o_time + (o_time_e - o_time_s)

    p_time_s = time.perf_counter()
    # -------- Penalized -------- (reweighting the train set)
    phi_train = esa_features[train_idx]
    tau_train = tau_values[train_idx]
    moral_penalty = np.array([(tau - esa_score(phi, alpha))**2 for phi, tau in zip(phi_train, tau_train)])
    sample_weights = np.clip(1 + 7 * moral_penalty, 1, 10)
    
    model_penalized = GaussianNB()
    model_penalized.fit(X_train, y_train, sample_weight=sample_weights)
    y_val_pred_penalized = model_penalized.predict(X_val)
    nb_penalized_preds[val_idx] = y_val_pred_penalized
    print(f"Fold {fold+1} - Naive Bayes ESA Penalized:")
    evaluate_classification(y_val, y_val_pred_penalized)
    p_time_e = time.perf_counter()
    p_time = p_time + (p_time_e - p_time_s)
    
print("\n--- Final Evaluation (Naive Bayes Baseline) ---")
evaluate_classification(y, nb_baseline_preds)
print("Accuracy:", accuracy_score(y, nb_baseline_preds))

print("\n--- Final Evaluation (Naive Bayes ESA Override) ---")
evaluate_classification(y, nb_override_preds)
print("Accuracy:", accuracy_score(y, nb_override_preds))

print("\n--- Final Evaluation (Naive Bayes ESA Penalized) ---")
evaluate_classification(y, nb_penalized_preds)
print("Accuracy:", accuracy_score(y, nb_penalized_preds))

print("time baseline:", b_time)
print("time override:", o_time)
print("time penalized:", p_time)

Fold 1 - Naive Bayes Baseline:

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90        65
           1       0.95      0.90      0.92        89

    accuracy                           0.92       154
   macro avg       0.91      0.92      0.91       154
weighted avg       0.92      0.92      0.92       154


Confusion Matrix:
[[61  4]
 [ 9 80]]
Fold 1 - Naive Bayes ESA Override:

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        65
           1       0.58      1.00      0.73        89

    accuracy                           0.58       154
   macro avg       0.29      0.50      0.37       154
weighted avg       0.33      0.58      0.42       154


Confusion Matrix:
[[ 0 65]
 [ 0 89]]
Fold 1 - Naive Bayes ESA Penalized:

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.

C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [12]:
metrics = evaluate_classification(y, nb_penalized_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.94      0.89       324
           1       0.95      0.88      0.91       445

    accuracy                           0.90       769
   macro avg       0.90      0.91      0.90       769
weighted avg       0.91      0.90      0.90       769


Confusion Matrix:
[[305  19]
 [ 55 390]]
{'accuracy': 0.9037711313394018, 'precision': 0.9535452322738386, 'recall': 0.8764044943820225, 'f1_score': 0.9133489461358314, 'roc_auc': 0.9088812595366902}


In [9]:
metrics = evaluate_classification(y, nb_override_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       324
           1       0.58      1.00      0.73       445

    accuracy                           0.58       769
   macro avg       0.29      0.50      0.37       769
weighted avg       0.33      0.58      0.42       769


Confusion Matrix:
[[  0 324]
 [  0 445]]
{'accuracy': 0.5786736020806242, 'precision': 0.5786736020806242, 'recall': 1.0, 'f1_score': 0.7331136738056013, 'roc_auc': 0.5}


C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\AISHA AIJAZ\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [13]:
metrics = evaluate_classification(y, nb_baseline_preds)
print(metrics)


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.94      0.90       324
           1       0.95      0.88      0.92       445

    accuracy                           0.91       769
   macro avg       0.90      0.91      0.91       769
weighted avg       0.91      0.91      0.91       769


Confusion Matrix:
[[305  19]
 [ 52 393]]
{'accuracy': 0.9076723016905072, 'precision': 0.9538834951456311, 'recall': 0.8831460674157303, 'f1_score': 0.9171528588098017, 'roc_auc': 0.9122520460535442}
